In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# %%
import torch
import string

In [3]:
!pip install sentencepiece

In [4]:
!pip install transformers

In [5]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [6]:
import pandas as pd
df = pd.read_csv("stockholm/mlm/new_literal.csv")

##BERT

In [7]:
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
cd drive/My Drive/Colab Notebooks/experiments

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/experiments'
/content/drive/My Drive/Colab Notebooks/experiments


In [9]:
# Use a subset for quick experiments
#data = data[:10000]

from sklearn.model_selection import train_test_split as tts
import pandas as pd
data = pd.read_csv("data/trofi.csv")

# Split to train, val and test
text, test_df = tts(data[["sentence", "verb", "label"]], random_state=42, test_size=0.1)
text, val = tts(text, random_state=42, test_size=test_df.shape[0])

In [10]:
text = text["sentence"]

In [11]:
#text.to_csv(r'mohx.txt', header=None, index=None, sep=' ', mode='a')

In [12]:
with open('data/trofi.txt', 'r') as fp:
    text = fp.read().split('\n')

In [13]:
text[:5]

['"So we \'re scoping the situation , kicking back , and all of a sudden this guy gets this insane barrel , then does a full - on airplane floater , then a full - on wraparound cutback , then a full - on snap reentry "',
 '"A group of history buffs in this border city is intent on seeing that Pancho Villa rests in peace rather than in pieces "',
 '"More recently , Pablo Casals \'s widow lent him her husband \'s precious Goffriller cello for two years and the teen - ager played chamber music at Carnegie Hall with Isaac Stern , Yo - Yo Ma and other luminaries "',
 '"We certainly do not need price limits ; Washington wo n\'t be any better at fixing stock prices than it was at fixing gasoline prices "',
 '"But the ANC has grown , and black supremacist breakaway groups have withered "']

In [14]:
inputs = tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [15]:
inputs

{'input_ids': tensor([[ 101, 1000, 2061,  ..., 5573, 2023,  102],
        [ 101, 1000, 1037,  ..., 9905, 6992,  102],
        [ 101, 1000, 2062,  ..., 2175, 4246,  102],
        ...,
        [ 101, 1000, 3130,  ..., 3274, 2449,  102],
        [ 101, 1000, 1011,  ..., 3778, 1005,  102],
        [ 101,  102,    0,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [16]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [17]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [18]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [19]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False,  True, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [20]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [21]:
selection[:5]

[[6, 8, 14], [11, 14, 16, 17], [1, 4, 11], [17, 19], [12]]

In [22]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [23]:
inputs.input_ids

tensor([[ 101, 1000, 2061,  ..., 5573, 2023,  102],
        [ 101, 1000, 1037,  ..., 9905, 6992,  102],
        [ 101,  103, 2062,  ..., 2175, 4246,  102],
        ...,
        [ 101, 1000, 3130,  ..., 3274, 2449,  102],
        [ 101, 1000, 1011,  ..., 3778, 1005,  102],
        [ 101,  102,    0,  ...,    0,    0,    0]])

In [24]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [25]:
dataset = MeditationsDataset(inputs)

In [26]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [27]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#model.to(device)

In [28]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [29]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/187 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 187/187 [06:49<00:00,  2.19s/it, loss=0.0839]


##XLM

In [30]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlmroberta_model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base')

In [31]:
inputs = xlmroberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [32]:
inputs

{'input_ids': tensor([[     0,     44,   8912,  ...,    756,    111,      2],
        [     0,     44,    284,  ...,   4378,   3089,      2],
        [     0,     44, 122880,  ...,    242,      7,      2],
        ...,
        [     0,     44, 235588,  ...,  21896,  17672,      2],
        [     0,     44,      9,  ...,  57622,    297,      2],
        [     0,      2,      1,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [33]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [34]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [35]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [36]:
mask_arr

tensor([[False,  True, False,  ..., False,  True, False],
        [False, False, False,  ..., False, False, False],
        [False,  True,  True,  ..., False, False, False],
        ...,
        [False, False, False,  ...,  True, False,  True],
        [False, False, False,  ..., False,  True, False],
        [False,  True, False,  ..., False, False,  True]])

In [37]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [38]:
selection[:5]

[[1, 12, 15, 19], [10], [1, 2, 3, 7, 10, 15], [12, 17], []]

In [39]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [40]:
inputs.input_ids

tensor([[     0,    103,   8912,  ...,    756,    103,      2],
        [     0,     44,    284,  ...,   4378,   3089,      2],
        [     0,    103,    103,  ...,    242,      7,      2],
        ...,
        [     0,     44, 235588,  ...,    103,  17672,    103],
        [     0,     44,      9,  ...,  57622,    103,      2],
        [     0,    103,      1,  ...,      1,      1,    103]])

In [41]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [42]:
dataset = MeditationsDataset(inputs)

In [43]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [44]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#xlmroberta_model.to(device)

In [45]:
from transformers import AdamW

# activate training mode
xlmroberta_model.train()
# initialize optimizer
optim = AdamW(xlmroberta_model.parameters(), lr=5e-5)

In [46]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = xlmroberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/187 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 187/187 [19:07<00:00,  6.13s/it, loss=0.218]


In [47]:
bert_model = model.eval()

In [48]:
xlmroberta_model = xlmroberta_model.eval()

##BART

In [49]:
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [ ]:
inputs = bart_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [ ]:
inputs

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [53]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [54]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [55]:
mask_arr

tensor([[False, False, False,  ...,  True, False, False],
        [False, False, False,  ..., False, False,  True],
        [False, False, False,  ..., False, False, False],
        ...,
        [False,  True, False,  ..., False,  True, False],
        [False, False, False,  ..., False, False, False],
        [False,  True,  True,  ..., False, False, False]])

In [56]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [57]:
selection[:5]

[[8, 14, 18], [10, 17, 20], [3], [9, 12, 15], [3, 19]]

In [58]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [59]:
inputs.input_ids

tensor([[    0,   113,  2847,  ...,   103,    42,     2],
        [    0,   113,   250,  ...,   139, 12470,   103],
        [    0,   113,  9690,  ..., 17483,   338,     2],
        ...,
        [    0,   103, 45095,  ...,  3034,   103,     2],
        [    0,   113,  5579,  ..., 12801,    53,     2],
        [    0,   103,   103,  ...,     1,     1,     1]])

In [60]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [61]:
dataset = MeditationsDataset(inputs)

In [62]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [63]:
from transformers import AdamW

# activate training mode
bart_model.train()
# initialize optimizer
optim = AdamW(bart_model.parameters(), lr=5e-5)

In [64]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = bart_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/187 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 187/187 [24:01<00:00,  7.71s/it, loss=0.294]


In [ ]:
bart_model = bart_model.eval()

##RoBERTa

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [ ]:
inputs = roberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [ ]:
inputs

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
inputs.keys()

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
selection[:5]

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
inputs.input_ids

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [80]:
from transformers import AdamW

# activate training mode
roberta_model.train()
# initialize optimizer
optim = AdamW(roberta_model.parameters(), lr=5e-5)

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = roberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/187 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 187/187 [07:46<00:00,  2.49s/it, loss=0.0755]


In [ ]:
roberta_model = roberta_model.eval()

##Masked Language Modeling Pipeline

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased').eval()

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [ ]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
electra_model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').eval()

In [ ]:
top_k = 10

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLNET LARGE =================================
    input_ids, mask_idx = encode(xlnet_tokenizer, text_sentence, False)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[:, :, mask_idx] = 1.0  # Previous tokens don't see last token
    target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
    target_mapping[0, 0, mask_idx] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

    with torch.no_grad():
        predict = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)[0]
    xlnet = decode(xlnet_tokenizer, predict[0, 0, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= BART =================================
    input_ids, mask_idx = encode(bart_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = bart_model(input_ids)[0]
    bart = decode(bart_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ELECTRA =================================
    input_ids, mask_idx = encode(electra_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = electra_model(input_ids)[0]
    electra = decode(electra_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ROBERTA =================================
    input_ids, mask_idx = encode(roberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = roberta_model(input_ids)[0]
    roberta = decode(roberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlnet': xlnet,
            'xlm': xlm,
            'bart': bart,
            'electra': electra,
            'roberta': roberta}

## after bart and roberta fine-tuning

In [ ]:
text_sentence = "The wildfire <mask> through the forest at an amazing speed."
get_all_predictions(text_sentence)

The wildfire <mask> through the forest at an amazing speed.


{'bart': 'raced\ntore\nburned\nripped\nswept',
 'bert': 'spread\nrolled\nripped\ntore\nburned',
 'electra': 'runs\npasses\nflows\nsweeps\nswept',
 'roberta': 'raced\nraged\nrolled\ntore\nsurged',
 'xlm': 'went\nmoved\npassed\nruns\nran',
 'xlnet': 'through\nwildfire\nfire\nroaring\ncircles'}

##after bart and roberta fine-tuning

In [ ]:
text_sentence = "The window panes were <mask> as the wind blew through them"
get_all_predictions(text_sentence)

The window panes were <mask> as the wind blew through them


{'bart': 'flying\ntorn\nfalling\ndestroyed\nsmashed',
 'bert': 'blackened\nblack\nshattered\nsmashed\ncharred',
 'electra': 'clear\nvisible\ncovered\nclosed\nopen',
 'roberta': 'shattered\ncracked\ncracking\ndamaged\nbroken',
 'xlm': 'closed\nlit\nflat\nopen\nsilent',
 'xlnet': 'as\njust\nwere\nsuch\nopen'}

## after bart and roberta fine-tuning

In [ ]:
text_sentence = "To heal and <mask> from death my heart"
get_all_predictions(text_sentence)

To heal and <mask> from death my heart


{'bart': 'escape\nreturn\nrelease\nrecover\nto',
 'bert': 'keep\nprotect\nrecover\nescape\nsafe',
 'electra': 'heal\nrecover\ndie\nlive\nsave',
 'roberta': 'protect\nkeep\ndefend\nsave\ndeliver',
 'xlm': 'free\nprotect\nsave\ndefend\nseparate',
 'xlnet': 'and\nfrom\ncome\nright\naway'}

In [ ]:
text_sentence ="The tax cut will <mask> the economy"
get_all_predictions(text_sentence)

The tax cut will <mask> the economy


{'bart': 'boost\naccelerate\nhelp\nstimulate\nrevive',
 'bert': 'boost\nhelp\nstimulate\ndamage\naid',
 'electra': 'hurt\nhelp\nboost\nbenefit\nimprove',
 'roberta': 'help\nboost\nstimulate\naid\nsupport',
 'xlm': 'boost\ncool\nhelp\nhurt\nbenefit',
 'xlnet': 'all\nwill\nbe\nnot\nthe'}

In [ ]:
text_sentence = "Black desert <mask> in iron silences"
get_all_predictions(text_sentence)

Black desert <mask> in iron silences


{'bart': 'animals\ncreatures\nwinds\nbeasts\nsand',
 'bert': 'sleeps\nians\nnights\ners\ndwellers',
 'electra': 'is\nappears\nflies\nes\nfalls',
 'roberta': 'bathing\nhides\ndances\nburns\nexists',
 'xlm': 's\nfilled\ncovered\nand\ned',
 'xlnet': 'and\nfor\nin\nis\non'}

##Wikipedia

In [ ]:
text_sentence = "0.000123, which  <mask> to a distance of 705 Mly, or 216 Mpc.000webhost is a free web hosting service"
get_all_predictions(text_sentence)

0.000123, which  <mask> to a distance of 705 Mly, or 216 Mpc.000webhost is a free web hosting service


{'bart': 'can\nflies\nrides\nextends\ntravels',
 'bert': 'translates\ncorresponds\nmeans\nextends\nrefers',
 'electra': 'extends\nruns\nvaries\napplies\noperates',
 'roberta': 'calculates\namounts\ntranslates\ncorresponds\nequals',
 'xlm': 'runs\npoints\nlives\nspread\nlinks',
 'xlnet': 'is\ncomes\ncame\nwas\nwent'}

In [ ]:
text_sentence = "operated by Hostinger.0010x0010 is a Dutch- <mask> audiovisual artist"
get_all_predictions(text_sentence)

operated by Hostinger.0010x0010 is a Dutch- <mask> audiovisual artist


{'bart': 'based\nbased\nregistered\nowned\nGerman',
 'bert': 'based\nborn\nspeaking\nlanguage\ncanadian',
 'electra': 'based\nborn\namerican\nlanguage\nowned',
 'roberta': 'born\nAmerican\nbased\nFrench\nGerman',
 'xlm': 'American\nborn\nbased\nbased\nlanguage',
 'xlnet': 'on\nmore\nthe\na\nnational'}

In [ ]:
text_sentence = "an alcohol abuse prevention program  <mask> in 2004 at Francis E. Warren Air Force Base"
get_all_predictions(text_sentence)

an alcohol abuse prevention program  <mask> in 2004 at Francis E. Warren Air Force Base


{'bart': 'started\nwas\nbegan\nlaunched\nopened',
 'bert': 'began\nstarted\nopened\nbegun\nestablished',
 'electra': 'began\nstarted\nlaunched\nopened\nestablished',
 'roberta': 'started\nestablished\nbeginning\nbegun\nbegan',
 'xlm': 'started\nbegan\ndeveloped\nestablished\napproved',
 'xlnet': 'in\nprogram\nbegan\nbeginning\nstarted'}

In [ ]:
text_sentence = "<mask> on research by the National Institute on Alcohol Abuse and Alcoholism regarding binge drinking in college students.0.01"
get_all_predictions(text_sentence)

<mask> on research by the National Institute on Alcohol Abuse and Alcoholism regarding binge drinking in college students.0.01


{'bart': "<s>\n</s>\nThe\n''\nthe",
 'bert': 'based\nreport\nreports\nresults\nagreement',
 'electra': 'based\ninformation\nreport\nfocusing\ndata',
 'roberta': 'Based\nbased\nbased\nBuilding\nDraw',
 'xlm': "Published\n''\nreported\nreport\ninformation",
 'xlnet': 'based\ninformation\nBased\ndata\nreport'}

In [ ]:
text_sentence = "is the debut studio album of H3llb3nt,  <mask> on February 20, 1996 by Fifth Colvmn Records.001 of 3 February 1997"
get_all_predictions(text_sentence)

is the debut studio album of H3llb3nt,  <mask> on February 20, 1996 by Fifth Colvmn Records.001 of 3 February 1997


{'bart': 'released\nand\nissued\npublished\nwhich',
 'bert': 'released\nissued\nrecorded\nrelease\nreleasing',
 'electra': 'released\nreissued\nrecorded\npublished\nrelease',
 'roberta': 'released\nrecorded\nissued\npublished\ndropped',
 'xlm': 'released\npublished\nrelease\nannounced\nout',
 'xlnet': 'released\nissued\npublished\nReleased\ndistributed'}

In [ ]:
text_sentence = " which was  <mask> between the Government of the Republic of Rwanda, and FAPADER.003230 is a South Korean food manufacturer."
get_all_predictions(text_sentence)

 which was  <mask> between the Government of the Republic of Rwanda, and FAPADER.003230 is a South Korean food manufacturer.


{'bart': 'established\ncreated\nformed\nfounded\na',
 'bert': 'agreed\nsigned\nsplit\nagreement\ncollaboration',
 'electra': 'negotiated\njointly\nshared\nagreed\nsplit',
 'roberta': 'negotiated\ncontracted\narranged\nagreed\nformed',
 'xlm': 'developed\nagreed\nagreement\nsigned\nbuilt',
 'xlnet': 'between\nthat\nmidway\nwas\nin'}

In [ ]:
text_sentence = "Gas molecules in soil are in continuous thermal motion  <mask> to the kinetic theory of gasses, there is also collision between molecules - a random walk.0.04"
get_all_predictions(text_sentence)

Gas molecules in soil are in continuous thermal motion  <mask> to the kinetic theory of gasses, there is also collision between molecules - a random walk.0.04


{'bart': 'and\nbecause\n--\nas\naccording',
 'bert': 'according\nsimilar\ncontrary\ndue\nthanks',
 'electra': 'according\nsimilar\ndue\ncompared\ncontrary',
 'roberta': 'According\naccording\nthanks\nThanks\nTrue',
 'xlm': 'according\nAccording\nthanks\ndue\nclose',
 'xlnet': 'according\nAccording\nAccording\ndue\naccording'}

In [ ]:
text_sentence = "is a first-person shooter video game <mask> the character of British secret agent James Bond."
get_all_predictions(text_sentence)

is a first-person shooter video game <mask> the character of British secret agent James Bond.


{'bart': 'featuring\nstarring\nabout\nbased\nthat',
 'bert': 'featuring\nabout\nstarring\ndepicting\nplaying',
 'electra': 'in\nfeaturing\nstarring\nwith\nabout',
 'roberta': 'featuring\ninvolving\nstarring\nabout\nin',
 'xlm': 'in\nfeaturing\nwith\nabout\nof',
 'xlnet': 'about\nwith\nfor\nfeaturing\non'}

In [ ]:
text_sentence = "007 Legends is the fourth and final James Bond game title  <mask> by Activision, the last game"
get_all_predictions(text_sentence)

007 Legends is the fourth and final James Bond game title  <mask> by Activision, the last game


{'bart': 'developed\npublished\nreleased\nproduced\nto',
 'bert': 'published\nreleased\nproduced\ndeveloped\nmade',
 'electra': 'owned\nreleased\nissued\nacquired\nsigned',
 'roberta': 'developed\nproduced\ncreated\npublished\nmade',
 'xlm': 'released\ndeveloped\nproduced\ncreated\npublished',
 'xlnet': 'title\nreleased\npublished\nplayed\nmade'}

In [ ]:
text_sentence = "<mask> on the James Bond license.00"
get_all_predictions(text_sentence)

<mask> on the James Bond license.00


{'bart': "<s>\n</s>\n''\nThe\nthe",
 'bert': 'based\nvariations\nriding\nclick\npassed',
 'electra': '00\n01\n50\n45\n30',
 'roberta': "''\n''\nBet\nPlaying\n--",
 'xlm': 'articles\nprices\nສໍານວນ\n</s>\ninsurance',
 'xlnet': 'on\non\nen\n00\n02'}

In [ ]:
text_sentence = "HappyPeeps.0.0 is a live album by Melt-Banana that  <mask> out on November 3, 2009.Melt"
get_all_predictions(text_sentence)

HappyPeeps.0.0 is a live album by Melt-Banana that  <mask> out on November 3, 2009.Melt


{'bart': 'came\ncomes\nrolled\nwent\nwas',
 'bert': 'came\nsold\ngot\nwent\nrolled',
 'electra': 'came\ncomes\ncome\nwent\nopened',
 'roberta': 'came\ncomes\nis\nwas\ndue',
 'xlm': 'came\nsteps\nis\nwent\ncomes',
 'xlnet': 'came\ncomes\nwas\nis\nwent'}

In [ ]:
text_sentence = "Melt-Banana Lite, which  <mask> that they  <mask> their music with a different arrangement"
get_all_predictions(text_sentence)

Melt-Banana Lite, which  <mask> that they  <mask> their music with a different arrangement


{'bart': 'says\nstates\nrecently\nannounced\nreports',
 'bert': 'claims\nindicates\nsays\nstates\nsaid',
 'electra': 'means\nsays\nshows\nensures\nsuggests',
 'roberta': 'means\nis\nsuggests\nnotes\nrequires',
 'xlm': 'means\nis\nshows\nfound\nsays',
 'xlnet': 'which\nthat\nmeans\nis\nthis'}

In [ ]:
text_sentence = ".0.0% is  <mask> to be the start of the retracement, while 100.0% is a complete reversal to the original part of the move"
get_all_predictions(text_sentence)

.0.0% is  <mask> to be the start of the retracement, while 100.0% is a complete reversal to the original part of the move


{'bart': 'believed\nconsidered\nwidely\nsupposed\nexpected',
 'bert': 'supposed\nsaid\nexpected\nassumed\nlikely',
 'electra': 'supposed\nexpected\nsaid\nlikely\ngoing',
 'roberta': 'likely\nbelieved\nsaid\nexpected\nconsidered',
 'xlm': 'said\nexpected\nlikely\nconsidered\ngoing',
 'xlnet': 'just\nnot\nsupposed\nintended\nset'}

In [ ]:
text_sentence = "00 pm in daily  <mask> on rerun programs to simultaneously networks.00 -"
get_all_predictions(text_sentence)

00 pm in daily  <mask> on rerun programs to simultaneously networks.00 -


{'bart': 'repeats\nrotation\nshows\nbroadcasts\nruns',
 'bert': 'and\nor\nsession\nbroadcasting\nbroadcast',
 'electra': 'updates\ntime\ntimes\ndays\nhours',
 'roberta': 'broadcasts\nbroadcast\ntrading\nrotation\nspecials',
 'xlm': 'data\nrates\nrate\nand\nrain',
 'xlnet': 'daily\non\nprogram\nprograms\ndaytime'}

In [ ]:
text_sentence = "The rebels  <mask> to  <mask> objects with a catapult."
get_all_predictions(text_sentence)

The rebels  <mask> to  <mask> objects with a catapult.


{'bart': 'were\nmanaged\nplan\nare\ntried',
 'bert': 'continued\nbegan\nused\ncontinue\nlearned',
 'electra': 'began\ntried\nproceeded\nattempted\nhad',
 'roberta': 'try\ntried\nattempt\naim\nprefer',
 'xlm': 'try\ntried\nattempt\nwant\nexpect',
 'xlnet': 'to\nagree\nare\non\ngo'}

In [ ]:
text_sentence = "01-04-043 as  <mask> in the catalog.010 being the areacode for the city of Rotterdam"
get_all_predictions(text_sentence)

01-04-043 as  <mask> in the catalog.010 being the areacode for the city of Rotterdam


{'bart': 'shown\nthe\nseen\ncited\ndisplayed',
 'bert': 'listed\nshown\ndescribed\nindicated\ndefined',
 'electra': 'shown\nincluded\nlisted\nspecified\ndescribed',
 'roberta': 'given\ndescribed\nrecorded\nnoted\nindicated',
 'xlm': 'shown\nstandard\ndetailed\nlisted\ndescribed',
 'xlnet': 'as\nused\nshown\ndescribed\nstated'}

In [ ]:
text_sentence = "01.10  <mask> to January 2010"
get_all_predictions(text_sentence)

01.10  <mask> to January 2010


{'bart': 'December\nto\n--\nSeptember\nJuly',
 'bert': 'up\naccounting\nthrough\n...\n2009',
 'electra': 'compared\ndates\nyear\nmarch\nprior',
 'roberta': 'prior\nthrough\nyears\nup\n--',
 'xlm': 'up\nUp\nGMT\nhours\nmonths',
 'xlnet': 'to\nand\nfrom\nin\ns'}

In [ ]:
text_sentence = "Revolution in Just  <mask>"
get_all_predictions(text_sentence)

Revolution in Just  <mask>


{'bart': 'icia\nFrance\nCauses\nWords\nBritain',
 'bert': 'days\nthat\nwords\nbeginning\nmonths',
 'electra': 'just\none\nright\n1\n2',
 'roberta': 'icia\nMadrid\nJazz\nPlain\nCity',
 'xlm': 'ice\nin\nification\none\nica',
 'xlnet': 'Just\nSo\nAnd\nU\nBut'}

In [ ]:
text_sentence = "01:59PM is the first studio album by South Korean boy band, 2PM.01708 is the national dialing code for Romford in the United Kingdom.0181 is a compilation album by Four Tet,  <mask> by Text Records"
get_all_predictions(text_sentence)

01:59PM is the first studio album by South Korean boy band, 2PM.01708 is the national dialing code for Romford in the United Kingdom.0181 is a compilation album by Four Tet,  <mask> by Text Records


{'bart': 'released\npublished\ndistributed\nproduced\nissued',
 'bert': 'released\ndistributed\nissued\npublished\nreissued',
 'electra': 'produced\nreleased\npublished\nand\ncompiled',
 'roberta': 'released\ndistributed\npublished\nproduced\ncompiled',
 'xlm': 'released\nproduced\npublished\nsigned\nand',
 'xlnet': 'released\npublished\nproduced\nissued\navailable'}

In [ ]:
text_sentence = "<mask> that the California Department of Corrections and Rehabilitation's (CDCR)"
get_all_predictions(text_sentence)

<mask> that the California Department of Corrections and Rehabilitation's (CDCR)


{'bart': "<s>\n</s>\nThe\nthe\n''",
 'bert': 'noting\nevidence\nand\nalleged\narguing',
 'electra': 'note\nnotice\nreports\nreport\nnotes',
 'roberta': 'Note\nExcept\nGiven\nConsider\nNow',
 'xlm': "''\n--\nAnd\nAccording\nNow",
 'xlnet': 'that\nis\ns\nof\nwas'}

In [ ]:
text_sentence = "medical services are inadequate and  <mask>"
get_all_predictions(text_sentence)

medical services are inadequate and  <mask>


{'bart': 'under\ncostly\noutdated\ntargeted\nneed',
 'bert': 'ineffective\nunreliable\ncostly\nexpensive\noutdated',
 'electra': 'inadequate\nexpensive\nineffective\nunnecessary\ncostly',
 'roberta': 'unavailable\ninefficient\nexpensive\nineffective\nunreliable',
 'xlm': 'expensive\ncheap\nslow\npoor\nmissing',
 'xlnet': 'are\nand\nthat\na\nis'}

In [ ]:
text_sentence = "<mask> in 2008.0.1% of all web pages  <mask> Windows-1254 in February 2016.01 "
get_all_predictions(text_sentence)

<mask> in 2008.0.1% of all web pages  <mask> Windows-1254 in February 2016.01 


{'bart': '<s>\n</s>\nThe\n0\n0',
 'bert': 'and\nup\nor\nbut\nfound',
 'electra': '1\n0\n01\n2\nposted',
 'roberta': 'Back\nBeginning\nStarting\nAlready\nPublished',
 'xlm': '<pad>\nמארטש\nΑναρτήσεις\nຮຽນພາສາອັງກິດ\nലൈഫ്',
 'xlnet': '54\n24\n56\n4\n04'}

In [ ]:
text_sentence = "<mask> in 2006 by Bom Kim and Daniel Loss, later  <mask> by Atlantic Media (David Bradley); it  <mask> graduates of Harvard University.021"
get_all_predictions(text_sentence)

<mask> in 2006 by Bom Kim and Daniel Loss, later  <mask> by Atlantic Media (David Bradley); it  <mask> graduates of Harvard University.021


{'bart': "<s>\n</s>\nThe\nIt\n''",
 'bert': 'founded\nlaunched\nstarted\nestablished\ncreated',
 'electra': 'published\nfounded\nlaunched\nreleased\ncreated',
 'roberta': 'Written\nPublished\nCreated\nMade\nReleased',
 'xlm': 'Published\npublished\ncreated\nproduced\ndeveloped',
 'xlnet': 'Incorporated\nproduced\nDistributed\npopulated\ncommissioned'}

In [ ]:
text_sentence = "<mask> from pGreenII 0229"
get_all_predictions(text_sentence)

<mask> from pGreenII 0229


{'bart': "<s>\n</s>\nThe\n''\nI",
 'bert': 'retrieved\nsourced\nquoting\ndata\nexcerpt',
 'electra': 'read\navailable\ndata\nvideo\noutput',
 'roberta': '--\nInformation\nQuestion\nDetails\ndescription',
 'xlm': '<pad>\nຮຽນພາສາອັງກິດ\nဓာတ်ပုံများ\nਤਸਵੀਰ\nΑναρτήθηκε',
 'xlnet': 'from\non\na\nor\nfar'}

In [ ]:
text_sentence = "9-time All-American while  <mask> for 4 years.02 9D 74 E3 5B D8 41"
get_all_predictions(text_sentence)

9-time All-American while  <mask> for 4 years.02 9D 74 E3 5B D8 41


{'bart': 'playing\nat\nrunning\ncoaching\nin',
 'bert': 'playing\nserving\nstarter\nthere\ninactive',
 'electra': 'serving\nplaying\nworking\nliving\nrunning',
 'roberta': 'playing\nthere\nmarried\nstarting\nbatting',
 'xlm': 'working\nplaying\nliving\nsitting\nmarried',
 'xlnet': 'for\nrunning\nplaying\nonly\njust'}

In [ ]:
text_sentence = "56 C5 63 56 88 C0 (commonly  <mask> to as 09 F9).02-cv-0299 is a habeas corpus petition"
get_all_predictions(text_sentence)

56 C5 63 56 88 C0 (commonly  <mask> to as 09 F9).02-cv-0299 is a habeas corpus petition


{'bart': 'referred\nreferring\nrefers\nrefer\ntranslated',
 'bert': 'referred\nrefers\nrefer\nabbreviated\nreferring',
 'electra': 'referred\nrefer\nknown\nspoken\ntranslated',
 'roberta': 'referred\nshortened\ngiven\nwritten\nReferred',
 'xlm': 'known\nreported\nagreed\ndescribed\nrefer',
 'xlnet': 'referred\nrefer\nreferring\nrefers\nknown'}

In [ ]:
text_sentence = "<mask> album, after Antisleep Vol.0.2% of the population"
get_all_predictions(text_sentence)

<mask> album, after Antisleep Vol.0.2% of the population


{'bart': "<s>\n</s>\nThe\n''\n0",
 'bert': 'the\nthis\nsecond\nthird\nfirst',
 'electra': 'the\nthis\nanother\nan\nhis',
 'roberta': 'The\nThis\nNew\nThat\nSecond',
 'xlm': "''\nThe\nThis\nNew\nsaid",
 'xlnet': 'the\nsoon\nyear\nalbum\nmajor'}

In [ ]:
text_sentence = "0.2% of the population  <mask> in the service industry and 0.1% works in the fishing"
get_all_predictions(text_sentence)

0.2% of the population  <mask> in the service industry and 0.1% works in the fishing


{'bart': 'works\nwork\nis\nworked\nearns',
 'bert': 'works\nis\nwork\nlives\nworked',
 'electra': 'works\nwork\nis\noperates\nplays',
 'roberta': 'works\nis\nparticipates\njobs\nlives',
 'xlm': 'lives\nis\nworks\ndies\njobs',
 'xlnet': 'works\nwork\nis\nlives\nare'}

In [ ]:
text_sentence = "industry.0.2%.The school was home to Detroit Pistons player Richard Rip Hamilton.03-04  <mask> on September 25, 2003"
get_all_predictions(text_sentence)

industry.0.2%.The school was home to Detroit Pistons player Richard Rip Hamilton.03-04  <mask> on September 25, 2003


{'bart': '--\nDetroit\n</s>\ncrash\nwas',
 'bert': 'students\nest\nclosed\nsemester\ndays',
 'electra': '–\nvictory\nearly',
 'roberta': 'died\nplayed\nDied\nbombed\nmet',
 'xlm': 'shooting\nand\nhours\n</s>\nClass',
 'xlnet': 'born\non\nreleased\nmade\n02'}

In [ ]:
text_sentence = "is an English-language, bi-annual contemporary culture magazine that  <mask> art, fashion, and politics.03 and the Blackstar score"
get_all_predictions(text_sentence)

is an English-language, bi-annual contemporary culture magazine that  <mask> art, fashion, and politics.03 and the Blackstar score


{'bart': 'examines\nexplores\ncovers\nfocuses\npresents',
 'bert': 'covers\nprofiles\ntargets\nincludes\nexamines',
 'electra': 'covers\npublishes\nfeatures\nexplores\ndiscusses',
 'roberta': 'covers\nexamines\nexplores\nexamined\nfeatures',
 'xlm': 'features\noffers\nreports\nstudies\nblogs',
 'xlnet': 'covers\npublishes\nincludes\nfeatures\ncover'}

In [ ]:
text_sentence = "03 is Isis's third live release.03 is the fourth album by Blue Stahli, and his third instrumental- <mask> album, after Antisleep Vol.0.3 mile south is"
get_all_predictions(text_sentence)

03 is Isis's third live release.03 is the fourth album by Blue Stahli, and his third instrumental- <mask> album, after Antisleep Vol.0.3 mile south is


{'bart': 'based\ndriven\noriented\nthemed\nsounding',
 'bert': 'instrumental\nbased\nrock\njazz\nonly',
 'electra': 'solo\ninstrumental\nrelease\nplay\ndriven',
 'roberta': 'rock\npop\nonly\ndriven\nband',
 'xlm': 'cover\nrock\nlong\nmetal\nelectric',
 'xlnet': 'based\nonly\noriented\nlength\nalbum'}

In [ ]:
text_sentence = "Thrumster Parish Church, part of the Church of Scotland charge of Pulteneytown and Thrumster.0.3% of ReadyReturn filings  <mask> errors, versus 3.1% of non-ReadyReturn filings.0.3 TBq is equal to 1/40th of the natural radiation in one km3 of seawater and 1/50000000000th of the total natural radioactivity in the oceans.04092001"
get_all_predictions(text_sentence)

Thrumster Parish Church, part of the Church of Scotland charge of Pulteneytown and Thrumster.0.3% of ReadyReturn filings  <mask> errors, versus 3.1% of non-ReadyReturn filings.0.3 TBq is equal to 1/40th of the natural radiation in one km3 of seawater and 1/50000000000th of the total natural radioactivity in the oceans.04092001


{'bart': 'have\nhad\nare\ncontain\nshow',
 'bert': 'report\ncontain\nhave\nshow\ncontained',
 'electra': 'are\nhave\ncontain\nwere\nreport',
 'roberta': 'contain\nmake\nare\nreport\ncontained',
 'xlm': 'were\nhad\nreported\nhave\nare',
 'xlnet': 'contain\nwere\ncontained\nare\nhad'}

In [ ]:
text_sentence = "is the fourth studio album by South Korean trio, Urban Zakapa,  <mask> of 9 track "
get_all_predictions(text_sentence)

is the fourth studio album by South Korean trio, Urban Zakapa,  <mask> of 9 track 


{'bart': 'composed\nconsisting\nand\nwith\ncomprised',
 'bert': 'consisting\ncomposed\nconsists\nconsisted\nconsist',
 'electra': 'consisting\ncomposed\nconsists\nout\nset',
 'roberta': 'consisting\ncomprised\ncomposed\nconsists\nfull',
 'xlm': 'out\ncollection\nset\nup\nmade',
 'xlnet': 'and\nout\nconsisting\nwhich\nas'}

In [ ]:
text_sentence = "<mask> on smaller frigates, so an alternative had to be sought.055 Brigade (or 55th Arab Brigade)"
get_all_predictions(text_sentence)

<mask> on smaller frigates, so an alternative had to be sought.055 Brigade (or 55th Arab Brigade)


{'bart': '<s>\n</s>\nThe\nthe',
 'bert': 'and\nbut\nor\nfiring\nespecially',
 'electra': 'especially\nnot\nbased\nbuilt\nparticularly',
 'roberta': 'fired\nflying\nfly\nfloat\nfly',
 'xlm': "''\n``\nworking\n«\nsleeping",
 'xlnet': 'operate\nused\nwork\nonly\nsailing'}

In [ ]:
text_sentence = "<mask> into the Taliban army between 1995 and 2001.0.5% calcium chlorideH6"
get_all_predictions(text_sentence)

<mask> into the Taliban army between 1995 and 2001.0.5% calcium chlorideH6


{'bart': "<s>\n</s>\nThe\n''\n0",
 'bert': 'inducted\ndrafted\nenlisted\nrecruited\nabsorbed',
 'electra': 'released\ndischarged\nrecruited\nincorporated\ndrafted',
 'roberta': 'Born\npoured\nforced\nFit\nMade',
 'xlm': "''\n\x95\n``\nBreaking\n\x93",
 'xlnet': 'into\nentry\nentered\nin\nre'}

In [ ]:
text_sentence = "<mask> in a number of military applications, notably underwater munitions (e.g. naval mines, depth charges and torpedoes)"
get_all_predictions(text_sentence)

<mask> in a number of military applications, notably underwater munitions (e.g. naval mines, depth charges and torpedoes)


{'bart': "<s>\n</s>\nThe\n''\nIt",
 'bert': 'and\nused\ninvolved\napplications\nengaged',
 'electra': 'used\nuseful\ninvolved\nspecialized\nengaging',
 'roberta': "and\n--\n''\nused\nUsed",
 'xlm': "``\nused\n''\n\x95\nand",
 'xlnet': 'used\nand\nas\nuse\nthem'}

In [ ]:
text_sentence = "where it has generally  <mask> torpex, being less shock sensitive and  "
get_all_predictions(text_sentence)

where it has generally  <mask> torpex, being less shock sensitive and  


{'bart': 'replaced\nsuppl\nbeen\noutper\nflourished',
 'bert': 'absorbed\nless\navoided\npreferred\nescaped',
 'electra': 'been\nless\nmore\nthe\na',
 'roberta': 'absorbed\nlacked\navoided\nbeen\nrequired',
 'xlm': 'killed\ntreated\nescape\nadded\nbeen',
 'xlnet': 'been\ngenerally\ntypically\nmost\nusually'}

In [ ]:
text_sentence = "<mask> more stable storage characteristics.0.5 cl is approximately 1 U.S."
get_all_predictions(text_sentence)

<mask> more stable storage characteristics.0.5 cl is approximately 1 U.S.


{'bart': "<s>\n</s>\nThe\n''\n0",
 'bert': 'for\nwith\nand\nbut\neven',
 'electra': 'much\nno\nfor\nwith\nthe',
 'roberta': 'Using\nThe\nWith\nBut\nFor',
 'xlm': "--\n''\n``\nFor\nwith",
 'xlnet': 'more\nand\nwith\nas\nin'}

In [ ]:
text_sentence = "Ross.061502 is a live concert DVD of Botch's final show at The Showbox in Seattle.0.6.17 of Skencil was  <mask> in 2005, and a preview 1.0 alpha was  "
get_all_predictions(text_sentence)

Ross.061502 is a live concert DVD of Botch's final show at The Showbox in Seattle.0.6.17 of Skencil was  <mask> in 2005, and a preview 1.0 alpha was  


{'bart': 'released\npublished\nproduced\nintroduced\nborn',
 'bert': 'released\nannounced\npublished\ncompleted\nleaked',
 'electra': 'released\nrecorded\nfilmed\nshown\nproduced',
 'roberta': 'recorded\nfilmed\nreleased\nshot\nmade',
 'xlm': 'released\nshown\nplayed\ncompleted\nout',
 'xlnet': 'released\nmade\nannounced\nproduced\npublished'}

In [ ]:
text_sentence = "in 2010.06 is a New Zealand compilation album  <mask>"
get_all_predictions(text_sentence)

in 2010.06 is a New Zealand compilation album  <mask>


{'bart': 'by\nreleased\nof\nfrom\nproduced',
 'bert': 'released\nrelease\nalbum\nissued\ncompilation',
 'electra': 'released\nalbum\nrelease\nrecord\ncompilation',
 'roberta': '\'\'\n""\nset\ncalled\nentitled',
 'xlm': 'called\nproduction\nand\nproduced\ndebut',
 'xlnet': 'album\nand\ncollection\nthat\nof'}

In [ ]:
text_sentence = "service is  <mask> on the principle of a flexible virtual telephone number.07548 and pin code is 473443.Ganesh Shankar Vidyarthi Indian independence movement activist and journalist  "
get_all_predictions(text_sentence)

service is  <mask> on the principle of a flexible virtual telephone number.07548 and pin code is 473443.Ganesh Shankar Vidyarthi Indian independence movement activist and journalist  


{'bart': 'based\nbuilt\nrun\noperated\nsold',
 'bert': 'based\noperated\nfocused\nbuilt\nfounded',
 'electra': 'based\nbuilt\nstructured\nprovided\noperated',
 'roberta': 'based\noperated\nbuilt\noperating\nfounded',
 'xlm': 'based\nbuilt\ndependent\noperating\ndesigned',
 'xlnet': 'based\nbuilt\noffered\navailable\noperating'}

In [ ]:
text_sentence = "Ganesh Shankar Vidyarthi Indian independence movement activist and journalist  <mask> here for sometime.07 Detva"
get_all_predictions(text_sentence)

Ganesh Shankar Vidyarthi Indian independence movement activist and journalist  <mask> here for sometime.07 Detva


{'bart': 'has\nworked\nlived\nwas\nhad',
 'bert': 'lived\nstayed\nresided\nworked\nsettled',
 'electra': 'is\nwas\nstays\nremains\nstayed',
 'roberta': 'is\nlives\nliving\nwas\nlived',
 'xlm': 'lives\nis\nlived\nwas\nworks',
 'xlnet': 'here\nright\nthere\naround\non'}

In [ ]:
text_sentence = "-8-4 originally  <mask> on ABC on October 1, 2013"
get_all_predictions(text_sentence)

-8-4 originally  <mask> on ABC on October 1, 2013


{'bart': 'aired\nran\nbroadcast\ndebuted\nappeared',
 'bert': 'aired\nbroadcast\nairing\npremiered\nair',
 'electra': 'aired\nappeared\npremiered\nannounced\nbroadcast',
 'roberta': 'aired\npremiered\nran\ndebuted\nbroadcast',
 'xlm': 'appeared\nreported\nran\nappears\nbegan',
 'xlnet': 'appeared\nappears\nreported\non\naired'}

In [ ]:
text_sentence = "North Rhine-Westphalia.08-cv-34, was a decision in the Northern District of Florida which  <mask> a student's First Amendment right to   "
get_all_predictions(text_sentence)

North Rhine-Westphalia.08-cv-34, was a decision in the Northern District of Florida which  <mask> a student's First Amendment right to   


{'bart': 'affirmed\nupheld\nasserted\nstruck\nattacked',
 'bert': 'affirmed\ndenied\nviolated\neliminated\nended',
 'electra': 'granted\nupheld\nincluded\nviolated\ngave',
 'roberta': 'affirmed\nprotects\nupheld\npreserved\nestablished',
 'xlm': 'increased\nallowed\nchanged\nprovided\nfixed',
 'xlnet': 'upheld\naffected\ngave\nestablished\ndeprived'}

In [ ]:
text_sentence = "Responding to pressure from the African-American community, the character Lil' Eightball was  <mask>"
get_all_predictions(text_sentence)

Responding to pressure from the African-American community, the character Lil' Eightball was  <mask>


{'bart': 'killed\ndropped\nremoved\nfired\nwritten',
 'bert': 'killed\ncreated\ndropped\nintroduced\nredesigned',
 'electra': 'born\nreleased\nintroduced\neliminated\nkilled',
 'roberta': 'created\ndeveloped\nredesigned\nraised\nadopted',
 'xlm': 'killed\nreleased\ncreated\nchanged\ncut',
 'xlnet': 'a\n“\nnamed\nwas\nnot'}

In [ ]:
text_sentence = "<mask> in a non-composited mode.0.9 is the fourth album by French rapper Booba"
get_all_predictions(text_sentence)

<mask> in a non-composited mode.0.9 is the fourth album by French rapper Booba


{'bart': "<s>\n</s>\nThe\n0\n''",
 'bert': 'recorded\nlive\nliving\nmusic\nplaying',
 'electra': 'released\nperformed\nrecorded\nwritten\nset',
 'roberta': 'But\nEven\nSometimes\nReleased\nWritten',
 'xlm': "''\nPublished\nלצפיה\n«\nproduced",
 'xlnet': 'in\nmode\nstate\nit\nsystem'}

In [ ]:
text_sentence = "Sri Lankans were eligible to  <mask> in the election.09 Towers is a complex of six towers"
get_all_predictions(text_sentence)

Sri Lankans were eligible to  <mask> in the election.09 Towers is a complex of six towers


{'bart': 'vote\nparticipate\ncast\ntake\nrun',
 'bert': 'vote\nrun\nstand\nparticipate\ncontest',
 'electra': 'vote\nparticipate\nrun\nwin\ncompete',
 'roberta': 'vote\nparticipate\nrun\ncompete\nstand',
 'xlm': 'vote\nbid\ncompete\ndie\nenter',
 'xlnet': 'vote\nparticipate\nin\nbe\nto'}

In [ ]:
text_sentence = "09 Towers is a complex of six towers  <mask> across a  built-up area of 450,000 square meters."
get_all_predictions(text_sentence)

09 Towers is a complex of six towers  <mask> across a  built-up area of 450,000 square meters.


{'bart': 'spread\nrising\nstretching\nbuilt\nscattered',
 'bert': 'spread\nstretching\nstretched\nscattered\nspreading',
 'electra': 'built\nrunning\nstretching\nlocated\nextending',
 'roberta': 'spread\nstretching\nscattered\nspaced\nspreading',
 'xlm': 'spread\nrunning\noperating\nlocated\nsleeping',
 'xlnet': 'spread\ntowers\nspanning\nset\nbased'}

In [ ]:
text_sentence = "squad.0 A.D. is a free and open-source, real-time strategy game under development by Wildfire Games.0DFx (also  <mask> as Zero Defects or Zero Defex) is an American hardcore punk band.0% financing (zero percent)"
get_all_predictions(text_sentence)

squad.0 A.D. is a free and open-source, real-time strategy game under development by Wildfire Games.0DFx (also  <mask> as Zero Defects or Zero Defex) is an American hardcore punk band.0% financing (zero percent)


{'bart': 'known\nreferred\nknow\nmarketed\ncommonly',
 'bert': 'known\nreleased\npublished\nmarketed\nsold',
 'electra': 'known\nclassified\nmarketed\nabbreviated\nlicensed',
 'roberta': 'known\nbranded\nmarketed\nknow\ncalled',
 'xlm': 'known\ndescribed\ncalled\nidentified\nregistered',
 'xlnet': 'known\nknow\navailable\nreleased\ncalled'}

In [ ]:
text_sentence = "is an American hardcore punk band.0% financing (zero percent), alternatively  <mask>"
get_all_predictions(text_sentence)

is an American hardcore punk band.0% financing (zero percent), alternatively  <mask>


{'bart': 'known\ncalled\navailable\nfinanced\nsold',
 'bert': 'called\ntitled\nknown\nwritten\nnamed',
 'electra': 'not\neither\nfunded\nsold\nincluded',
 'roberta': 'said\n1\nsays\n2\ncalculated',
 'xlm': 'affordable\nwith\noperating\navailable\noffered',
 'xlnet': 'to\nthat\ncalled\nfrom'}

In [ ]:
text_sentence = "marketing tactic for  <mask> buyers of consumer goods, automobiles, real estate, or credit cards in different parts of the world.0 has been  <"
get_all_predictions(text_sentence)

marketing tactic for  <mask> buyers of consumer goods, automobiles, real estate, or credit cards in different parts of the world.0 has been  <


{'bart': 'potential\ntargeting\ntargeted\nwoo\nthe',
 'bert': 'prospective\nblack\ncaptive\nyoung\nboth',
 'electra': 'the\nall\nmany\ndifferent\nboth',
 'roberta': 'overseas\nforeign\npotential\nprospective\naffluent',
 'xlm': 'regular\nfrequent\nmost\nmany\nthe',
 'xlnet': 'the\nvarious\ncommercial\npotential\nprospective'}

In [ ]:
text_sentence = "that  <mask> place near the Kilometer Zero marker in the Puerta del Sol in Madrid.0 is the best quality"
get_all_predictions(text_sentence)

that  <mask> place near the Kilometer Zero marker in the Puerta del Sol in Madrid.0 is the best quality


{'bart': 'took\ntakes\ntake\nceremony\nstatue',
 'bert': 'one\ntakes\ntook\npolling\na',
 'electra': 'in\ntakes\nexact\nthe\none',
 'roberta': 'takes\ntaking\ntook\ntake\nits',
 'xlm': 'takes\ntook\ntake\ntaking\nits',
 'xlnet': 'take\ntakes\ntook\nthey\nthat'}

In [ ]:
text_sentence = "0music is the second album  <mask> with Melomics technology."
get_all_predictions(text_sentence)

0music is the second album  <mask> with Melomics technology.


{'bart': 'produced\ndeveloped\nmade\nrecorded\nreleased',
 'bert': 'recorded\nmade\nbuilt\ndesigned\ncreated',
 'electra': 'released\nrecorded\npublished\nmade\ndone',
 'roberta': 'made\ncreated\nrecorded\nproduced\ndeveloped',
 'xlm': 'produced\ndeveloped\nreleased\ncreated\ndesigned',
 'xlnet': 'made\nalbum\nwith\nout\nreleased'}

In [ ]:
text_sentence = "<mask> for the operator for trunk calls before subscriber trunk dialing (STD) was  <mask> , and so was  "
get_all_predictions(text_sentence)

<mask> for the operator for trunk calls before subscriber trunk dialing (STD) was  <mask> , and so was  


{'bart': "<s>\n</s>\nThe\nIt\n''",
 'bert': 'as\nwaiting\nfortunately\nbut\naccounting',
 'electra': 'waiting\ntime\ncall\nonly\nwait',
 'roberta': 'Testing\nSupport\nControl\nRegistration\nEasy',
 'xlm': "''\nCompetition\n``\n--\nWork",
 'xlnet': 'was\nand\nmade\ncalled\nbe'}

In [ ]:
text_sentence = "0x10C is a hexadecimal number equivalent to 1612 in decimal, which  <mask> 281,474,976,710,656,"
get_all_predictions(text_sentence)

0x10C is a hexadecimal number equivalent to 1612 in decimal, which  <mask> 281,474,976,710,656,


{'bart': 'is\nequals\ntotals\nweighs\nhas',
 'bert': 'includes\nmeans\nis\nrepresents\ncomprises',
 'electra': 'is\nincludes\nmeans\ncomprises\nare',
 'roberta': 'equals\nrepresents\ntotals\nis\nyields',
 'xlm': 'is\nincludes\nhas\ncosts\nmeans',
 'xlnet': 'includes\ninclude\nis\nnumber\nwith'}

In [ ]:
text_sentence = "a boxed set <mask> in 2004 on Island Records.$1,000,000 Chance of a Lifetime is an American game show"
get_all_predictions(text_sentence)

a boxed set <mask> in 2004 on Island Records.$1,000,000 Chance of a Lifetime is an American game show


{'bart': 'was\nof\nreleased\ncame\narrived',
 'bert': 'released\nissued\nappeared\nreissued\npublished',
 'electra': 'released\ndebuted\npremiered\nset\nrecorded',
 'roberta': 'released\nrecorded\npublished\nset\nmade',
 'xlm': 'released\npublished\nrelease\nup\nout',
 'xlnet': 'set\nin\nreleased\nsold\nbox'}

In [ ]:
text_sentence = "1982 SH1, is a Hungaria asteroid from the innermost region of the asteroid belt.100,000 Cobbers is a 1942  <mask> documentary"
get_all_predictions(text_sentence)

1982 SH1, is a Hungaria asteroid from the innermost region of the asteroid belt.100,000 Cobbers is a 1942  <mask> documentary


{'bart': 'Soviet\nBritish\nDutch\nHungarian\nFrench',
 'bert': 'british\nsoviet\namerican\nshort\nwar',
 'electra': 'nasa\namerican\ncanadian\ndocumentary\nbritish',
 'roberta': 'Japanese\ntelevision\nGerman\nHolocaust\nSoviet',
 'xlm': 'action\nshort\nfilm\nexperimental\nenvironmental',
 'xlnet': 'documentary\ntelevision\nBritish\nAustralian\nfilm'}

In [ ]:
text_sentence = "100.000 dollari per Ringo is a 1965 spaghetti western film <mask> by Alberto De Martino.100000 faction in water is the most attractive, "
get_all_predictions(text_sentence)

100.000 dollari per Ringo is a 1965 spaghetti western film <mask> by Alberto De Martino.100000 faction in water is the most attractive, 


{'bart': 'directed\nwritten\nproduced\nstarring\nby',
 'bert': 'directed\nwritten\nproduced\nscored\nmade',
 'electra': 'directed\nproduced\nwritten\nmade\nadapted',
 'roberta': 'directed\nmade\nproduced\nwritten\ncreated',
 'xlm': 'ed\nwritten\nmade\nplayed\nproduced',
 'xlnet': 'directed\nmade\nfilm\nproduced\nwritten'}

In [ ]:
text_sentence = "then mosquitoes are  <mask> instead.100.000 of niets (100,000 or nothing)"
get_all_predictions(text_sentence)

then mosquitoes are  <mask> instead.100.000 of niets (100,000 or nothing)


{'bart': 'attacking\nbiting\nkilled\nflying\neating',
 'bert': 'killed\nused\nattacked\nplanted\ntargeted',
 'electra': 'fed\nthere\nintroduced\ncaught\ninfected',
 'roberta': 'killed\nattacked\ntargeted\nmultiplying\nattacking',
 'xlm': 'killed\nsleeping\neating\nallowed\ncaught',
 'xlnet': 'instead\nnot\nthere\nfirst\nthen'}

In [ ]:
text_sentence = "was a Flemish TV quiz ,  <mask> on the Flemish public service TV station N.I.R. (nowadays the VRT), between 1956 and 1959 on Sunday evenings.100,000 (one hundred thousand)"
get_all_predictions(text_sentence)

was a Flemish TV quiz ,  <mask> on the Flemish public service TV station N.I.R. (nowadays the VRT), between 1956 and 1959 on Sunday evenings.100,000 (one hundred thousand)


{'bart': 'broadcast\nshown\naired\nwhich\npresented',
 'bert': 'broadcast\naired\nshown\nairing\nrepeated',
 'electra': 'broadcast\naired\nconducted\nairing\nheld',
 'roberta': 'broadcast\nshown\naired\nscreened\ncarried',
 'xlm': 'broadcast\nplayed\nran\nshown\nset',
 'xlnet': 'broadcast\naired\nshown\nalso\nrun'}

In [ ]:
text_sentence = "is the natural number  <mask> 99,999 and preceding 100,001.100, 000/-"
get_all_predictions(text_sentence)

is the natural number  <mask> 99,999 and preceding 100,001.100, 000/-


{'bart': 'after\npreceding\nabove\nof\nbefore',
 'bert': 'preceding\nfollowing\nafter\nbetween\nbefore',
 'electra': 'preceding\nof\nfollowing\nafter\nabove',
 'roberta': 'of\nafter\nbetween\n≥\npreceding',
 'xlm': 'of\nabove\nbetween\nbeing\nbefore',
 'xlnet': '99\nand\nof\nnumber\nnow'}

In [ ]:
text_sentence = "(Sri Lankan Rupees) for anyone who  could  <mask> supernatural or miraculous powers under fool-proof and fraud-proof conditions"
get_all_predictions(text_sentence)

(Sri Lankan Rupees) for anyone who  could  <mask> supernatural or miraculous powers under fool-proof and fraud-proof conditions


{'bart': 'show\nclaim\ndisplay\ndemonstrate\nprove',
 'bert': 'manifest\nhave\nperform\nuse\ndemonstrate',
 'electra': 'possess\nhave\nuse\ngain\ndevelop',
 'roberta': 'have\npossess\nclaim\nobtain\nachieve',
 'xlm': 'escape\nexploit\nfind\nobtain\nenter',
 'xlnet': 'supernatural\nhave\nperform\nmiraculous\nmake'}

In [ ]:
text_sentence = "Gujarat Police.10,000 acres (40 km2) of  <mask> > land"
get_all_predictions(text_sentence)

Gujarat Police.10,000 acres (40 km2) of  <mask> > land


{'bart': 'forest\nconfiscated\ncultivated\nwater\nraw',
 'bert': 'farm\npaddy\ntrust\nforested\ngovt',
 'electra': 'land\nforest\npublic\ntar\na',
 'roberta': 'forest\nagricultural\nagriculture\nfarm\nfarmland',
 'xlm': 'farm\nagricultura\ncommercial\nmature\norganic',
 'xlnet': 'of\nthe\nits\nall\nhis'}

In [ ]:
text_sentence = "Black Men Named George is "
get_all_predictions(text_sentence)

Black Men Named George is 


IndexError: ignored

In [ ]:
text_sentence = "Ranchers Bees Stadium is their home venue.10,000 copies were  <mask> "
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "1.10,000 km is a Spanish romantic drama film  <mask> by Carlos Marques-Marcet."
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "Zeni Geva.10.000 luchtballonnen is the fourteenth studio album by Belgian-Dutch girl group K3.10,000 Maniacs in 2001.It was quickly a sextet, which  <mask> Ryan Seekings on drums, Stan Barton on fiddle,"
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "10,000 Maniacs is an American alternative rock band that was  <mask> in 1981.10,000 maps 4,000 musical scriptsThe research library today has approximately 850,000 volumes with collection emphasis on the German literature."
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "The research library today has approximately 850,000 volumes with collection emphasis on the German literature.10000 mEach event is <mask> according to the decathlon  <mask> tables or, for non-decathlon events, "
get_all_predictions(text_sentence)